In [4]:
# !pip install tensorflow
# !pip install keras


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint
# fix random seed for reproducibility
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')

In [11]:
import zipfile

In [12]:
zf = zipfile.ZipFile('./Dataset/consumer_complaints.csv.zip')
df = pd.read_csv(zf.open('consumer_complaints.csv'))

In [15]:
Data = df[['product','consumer_complaint_narrative']]
Data = Data[pd.notnull(Data['consumer_complaint_narrative'])]
Data


product                       consumer_complaint_narrative
190126   Debt collection  XXXX has claimed I owe them {$27.00} for XXXX ...
190135     Consumer Loan  Due to inconsistencies in the amount owed that...
190155          Mortgage  In XX/XX/XXXX my wages that I earned at my job...
190207          Mortgage  I have an open and current mortgage with Chase...
190208          Mortgage  XXXX was submitted XX/XX/XXXX. At the time I s...
...                  ...                                                ...
553084  Credit reporting  XXXX XXXX is reporting incorrectly, payments h...
553085  Credit reporting  Reflecting incorrect payment status. Have been...
553086       Payday loan  I have been paying {$180.00} a month through d...
553090          Mortgage  I recently became aware that Amerisave Mortgag...
553096          Mortgage  Bank of America has demonstrated an on-going l...

[66806 rows x 2 columns]

In [17]:
X=Data['consumer_complaint_narrative']
y=Data['product']

In [18]:
X.shape

(66806,)

In [18]:
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,test_size = 0.2)
print("Shape of train data:", X_train.shape)
print("Shape of Test data:", X_test.shape)
print("Shape of CV data:", X_cv.shape)

# truncate and pad input sequences
max_review_length = 200
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
X_cv = sequence.pad_sequences(X_cv,maxlen=max_review_length)

Shape of train data: (20000,)
Shape of Test data: (25000,)
Shape of CV data: (5000,)


In [21]:
# create the model
embedding_vecor_length = 16
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
filepath="weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, y_train, epochs=5, batch_size=256,verbose = 1,callbacks = callbacks_list,validation_data=(X_cv,y_cv))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 200, 16)           160000    
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 206,901
Trainable params: 206,901
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
79/79 [==============================] - 43s 509ms/step - loss: 0.6641 - accuracy: 0.5783 - val_loss: 0.4565 - val_accuracy: 0.8030

Epoch 00001: val_accuracy improved from -inf to 0.80300, saving model to weights_best.hdf5
Epoch 2/5
79/79 [==============================] - 50s 638ms/step - loss: 0.3289 - accuracy: 0.8694 - val_loss: 0.3493 - val_accuracy: 0.87

In [1]:

# Final evaluation of the model
embedding_vecor_length = 16
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.load_weights("weights_best.hdf5")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
scores = model.evaluate(X_test, y_test, verbose=1,batch_size = 256)
print("Accuracy: %.2f%%" % (scores[1]*100))

NameError: name 'Sequential' is not defined

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
filepath="weights_best_cnn.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, y_train, epochs=5, batch_size=256,verbose = 1,callbacks = callbacks_list,validation_data=(X_cv,y_cv))

In [ ]:
# Final evaluation of the model
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.load_weights("weights_best_cnn.hdf5")
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:

table = PrettyTable()
table.field_names = ['Model', 'Accuracy']
table.add_row(['LSTM', 85.50])
table.add_row(['CNN using LSTM', 87.50])
print(table)